### https://www.kaggle.com/code/deepak180222/question-answering-using-bloom-7b1-and-chromadb

In [1]:
! nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-cce78278-2d3b-2792-b7bf-b8df28299318)


In [2]:
%%time

! pip install -qq -U langchain tiktoken pypdf chromadb faiss-gpu
! pip install -qq -U transformers InstructorEmbedding sentence_transformers

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
google-cloud-pubsublite 1.8.2 requires overrides<7.0.0,>=6.0.1, but you have overrides 7.4.0 which is incompatible.
jupyterlab-lsp 4.2.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
ray 2.5.1 requires grpcio<=1.51.3,>=1.42.0; python_version >= "3.10" and sys_platform != "darwin", but you have grpcio 1.59.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.4.14 requires grpcio>=1.58.0, but you have grpcio 1.51.3 which is inco

In [3]:
import os
import textwrap
import glob
import shutil

import langchain
from langchain.llms import HuggingFacePipeline
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

from langchain.vectorstores import Chroma, FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA, VectorDBQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

/opt/conda/lib/python3.10/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
#os.makedirs('/kaggle/working/mk-vectordb-chroma')

In [5]:
#shutil.rmtree('/kaggle/working/mk-vectordb-chroma',ignore_errors=True)

In [6]:
#shutil.copytree('/kaggle/input/mkvc-2-part-1/kaggle/working/mk-vectordb-chroma-two', '/kaggle/working/mk-vectordb-chroma-two')

In [7]:
#shutil.rmtree('/kaggle/working/dataset',ignore_errors=True)

In [8]:
#os.makedirs('/kaggle/working/dataset')

In [9]:
#os.remove('/kaggle/working/file.zip')

In [10]:
%%time
from sentence_transformers import SentenceTransformer, models
#from chromadb import Documents, EmbeddingFunction, Embeddings
#instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="LazarusNLP/simcse-indobert-lite-base", model_kwargs={"device": "cuda"})
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="/kaggle/input/mkbertfx",model_kwargs={"device": "cuda"})

load INSTRUCTOR_Transformer
max_seq_length  512
CPU times: user 4.3 s, sys: 1.31 s, total: 5.61 s
Wall time: 11.7 s


In [11]:
noke = 0
based_ke_from = 0
actual_from = 0
actual_to = 0

for x in range(40):
    noke += 1
    
    actual_from = based_ke_from+1
    actual_to = based_ke_from+100 
    
    print(noke,actual_from," ~ ",actual_to)
    based_ke_from = actual_to
    
    shutil.rmtree('/kaggle/working/dataset',ignore_errors=True)
    os.makedirs('/kaggle/working/dataset')
    
    #Pindahkan 100
    files = glob.glob("/kaggle/input/mkri-fullset/*")
    ke = 0
    for file in files:
        ke += 1
        if ke >= actual_from and ke <= actual_to:
            shutil.copy(file, '/kaggle/working/dataset/')
            
    print(len(glob.glob('/kaggle/working/dataset/*')))
    
    loader = DirectoryLoader('/kaggle/working/dataset/', 
                             glob="./*.pdf",
                             loader_cls=PyPDFLoader,
                             show_progress=True,
                             use_multithreading=True)

    documents = loader.load()
    
    ### clean a bit
    for i in range(len(documents)):
        documents[i].page_content = documents[i].page_content.replace('\t', ' ')\
                                                             .replace('\n', ' ')\
                                                             .replace('       ', ' ')\
                                                             .replace('      ', ' ')\
                                                             .replace('     ', ' ')\
                                                             .replace('    ', ' ')\
                                                             .replace('   ', ' ')\
                                                             .replace('  ', ' ')
        
    print(len(documents))
    
    #print(documents[30].page_content)
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    print(len(texts))
    
    persist_directory = 'mk-vectordb-chroma-twenty'

    ### create embeddings and DB
    vectordb = Chroma.from_documents(documents=texts,
                                     embedding=instructor_embeddings,
                                     persist_directory=persist_directory,
                                     collection_name='mk_docs')


    ### persist Chroma database
    vectordb.persist()

1 1  ~  100
100


100%|██████████| 100/100 [06:37<00:00,  3.98s/it]


6761
16053
2 101  ~  200
100


100%|██████████| 100/100 [09:36<00:00,  5.77s/it]


8732
20912
3 201  ~  300
100


...

8764
21540
39 3801  ~  3900
100


100%|██████████| 100/100 [06:20<00:00,  3.81s/it]


7445
17887
40 3901  ~  4000
94


100%|██████████| 94/94 [06:40<00:00,  4.26s/it]


6862
17065


In [12]:
%%time

!zip -r file.zip /kaggle/working/mk-vectordb-chroma-twenty

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: kaggle/working/mk-vectordb-chroma-twenty/ (stored 0%)
  adding: kaggle/working/mk-vectordb-chroma-twenty/chroma.sqlite3 (deflated 45%)
  adding: kaggle/working/mk-vectordb-chroma-twenty/795e5318-7c71-4aa8-849e-6715cb741e3b/ (stored 0%)
  adding: kaggle/working/mk-vectordb-chroma-twenty/795e5318-7c71-4aa8-849e-6715cb741e3b/length.bin (deflated 67%)
  adding: kaggle/working/mk-vectordb-chroma-twenty/795e5318-7c71-4aa8-849e-6715cb741e3b/header.bin (deflated 52%)
  adding: kaggle/working/mk-vectordb-chroma-twenty/795e5318-7c71-4aa8-849e-6715cb741e3b/data_level0.bin (deflated 9%)
  adding: kaggle/working/mk-vectordb-chroma-twenty/795e5318-7c71-4aa8-849e-6715cb741e3b/index_metadata.pickle (deflated 81%)
  adding: kaggle/working/mk-vectordb-chroma-twenty/795e5318-7c71-4aa8-849e-6715cb741e3b/link_lists.bin (deflated 67%)
CPU times: user 8.57 s, sys: 1.8 s, total: 10.4 s
Wall time: 8min 57s


In [13]:
#shutil.rmtree('/kaggle/working/dataset',ignore_errors=True)
#os.remove('/kaggle/working/file.zip')